<a href="https://colab.research.google.com/github/raquelaoki/DataAnalysis2016/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##RNN

These are a type of Neural Networks (NN) that can have a long or shor 'memory'. In a scenario where the sequence of interest has $K$ elements/time stemps, the NN will have $K$ layers, one for each element. Considering a folded RNN format, there are 3 nodes: $x_t$ (input at step $t$), $s_t$ (hidden state at time $t$, the 'memory', $s_t = f(U\times x_t+W\times s_{t-1}))$ and $o_t$ (the output on the state $t$, $o_t = actfunc(V\times s_t)$). A RNN share the parameters $U$, $V$ and $W$ across all steps.

There are some variations that can be adopted: one-to-many (Image Description), many-to-one (sentiment analysis), many-to-many (Text Generation, Translation). 

###LSTM 

Long Short Term Memory (LSTM) are a subtype of RNN, design for long-term dependences. While RNN can also support long-term dependences, on its pure form, has a poor performance and struggle to connect past information. A LSTM cell has gates, that let the information go through or holds information, usually a sigmoid neural net. A LSTM has 3 of these gates: 
1. Forget gate layer: considering $h_{t-1}$ and $x_t$, and outputs a values between 0 and 1 for each number in the cell state $C_{t-1}$. Values near to 1 means that the values are kept, and 0 that it should get rid of it. $f_t = \sigma(W_f.[h_{t-1},x_t]+b_f)$
2. Decide if the information will be store in the cell. We do this in two parts: first, we use a input gate layer to decide which values to update. Then, a new layer calculate the new candidate values. 
$i_t=\sigma(W_i.[h_{t-1},x_t]+b_i)$ and $\hat{C}_t = tann(W_c.[h_{t-1},x_t]+b_c)$
3. The cell is update by combine the values previous calculated: $C_t=f_t*C_{t-1}+i_t*\hat{C_t}$. The output will be $o_t=\sigma(W_o [h_{t-1}, x_t]+b_o)$ and the new hidden state $h_t = o_t*tanh(C_t)$.




References:
- RNN 
 - [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
 - [RNN and Python Example](https://towardsdatascience.com/recurrent-neural-networks-by-example-in-python-ffd204f99470)
 - [Tutorial in lua 1](https://github.com/jcjohnson/torch-rnn)
 - [Tutorial in lua 1](https://github.com/karpathy/char-rnn/blob/master/model/LSTM.lua)

- LSTM 
 - [Understanding LSTM](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) 
 - [Illustrated Guide to LSTM’s and GRU’s: A step by step explanation](https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21)
  - [Fall of RNN/LSTM](https://towardsdatascience.com/the-fall-of-rnn-lstm-2d1594c74ce0)
  - [How to Develop LSTM Models for Time Series Forecasting](https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/)
 - [Tutorial for LSTM - Tensorflow 1](https://adventuresinmachinelearning.com/recurrent-neural-networks-lstm-tutorial-tensorflow/)
 - [Tutorial for RNN/LSTM - Tensorflow 2](https://github.com/dragen1860/TensorFlow-2.x-Tutorials/blob/master/09-RNN-Sentiment-Analysis/main.py)




Note: Tensorflow 2



In [0]:
#!pip uninstall tensorflow -y
#!pip uninstall tf-nightly -y
#!pip uninstall tf-nightly-gpu -y 
#!pip install tensorflow-gpu==2.0.0 


In [0]:
import  os
import  tensorflow as tf
import  numpy as np
from    tensorflow import keras

#Using the code from available in:
#https://github.com/dragen1860/TensorFlow-2.x-Tutorials/blob/master/09-RNN-Sentiment-Analysis/main.py
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.__version__

In [0]:
#what does os.environ? 
#fix random seed for reproducibility and checking tensor version
tf.random.set_seed(22)
np.random.seed(22)
assert tf.__version__.startswith('2.')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'



In [0]:
#TEST WITH A DIFFERENT DATASET
#while its a functional exemple, does not show the generated text back 



# load the dataset but only keep the top n words, zero the rest
top_words = 10000
# truncate and pad input sequences
max_review_length = 80
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=top_words)
# X_train = tf.convert_to_tensor(X_train)
# y_train = tf.one_hot(y_train, depth=2)
print('Pad sequences (samples x time)')
x_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_review_length)
x_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_review_length)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [0]:
class RNN(keras.Model):

    def __init__(self, units, num_classes, num_layers):
        super(RNN, self).__init__()

        # self.cells = [keras.layers.LSTMCell(units) for _ in range(num_layers)]
        #
        # self.rnn = keras.layers.RNN(self.cells, unroll=True)
        self.rnn = keras.layers.LSTM(units, return_sequences=True)
        self.rnn2 = keras.layers.LSTM(units)
        # self.cells = (keras.layers.LSTMCell(units) for _ in range(num_layers))
        # #
        # self.rnn = keras.layers.RNN(self.cells, return_sequences=True, return_state=True)
        # self.rnn = keras.layers.LSTM(units, unroll=True)
        # self.rnn = keras.layers.StackedRNNCells(self.cells)

        # have 1000 words totally, every word will be embedding into 100 length vector
        # the max sentence lenght is 80 words, top_words = 10000
        self.embedding = keras.layers.Embedding(top_words, 100, input_length=max_review_length)
        self.fc = keras.layers.Dense(1)

    def call(self, inputs, training=None, mask=None):

        # print('x', inputs.shape)
        # [b, sentence len] => [b, sentence len, word embedding]
        x = self.embedding(inputs)
        # print('embedding', x.shape)
        x = self.rnn(x) 
        x = self.rnn2(x) 
        # print('rnn', x.shape)

        x = self.fc(x)
        print(x.shape)

        return x

In [0]:
#def main():

units = 64
num_classes = 2
batch_size = 32
epochs = 20

model = RNN(units, num_classes, num_layers=2)

model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# train
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
          validation_data=(x_test, y_test), verbose=1)

# evaluate on test set
scores = model.evaluate(x_test, y_test, batch_size, verbose=1)
print("Final test loss and accuracy :", scores)

